# MaDDash and central esmond Measurement Archive
<p>The following steps will install the `perfsonar-centralmanagement` bundle to provide a standalone MaDDash server and central esmond Measurement Archive (MA)</p>

<p>At the end of the session we will have a central MA to which to register GridFTP and perfSONAR test results and dashboards to visualize performance:</p>
<ul>
<li>initial system setup, base installation, and initial configuration</li>
<li>build and publish a mesh json for consumption by the MaDDash server and participating perfSONAR nodes</li>
<li>update the MaDDash configuration for GridFTP</li>
<li>establish credentials for nodes to use in registering GridFTP and perfSONAR results</li>
</ul>

## pull in the perfsonar-centralmanagement notebook
<p>using your terminal app log into your fionette as root, and cd into /root</p>
<p>use wget to transfer the perfsonar-centralmanagement notebook<pre>
wget http://fiona-10-01:8000/perfsonar-centralmanagement.ipynb -O perfsonar-centralmanagement.ipynb
</pre>
<p>launch jupyter lab</p>
<pre>jupyter lab --allow-root --ip=`your.fionette.ip`</pre>

## initial system setup

In [4]:
uname -r

3.10.0-693.17.1.el7.x86_64


In [5]:
sed -i 's/SELINUX=enforcing/SELINUX=disabled/g' /etc/selinux/config
setenforce 0

setenforce: SELinux is disabled


: 1

## install NTP, bind-utils, traceroute if not already present

In [6]:
yum -y install ntp bind-utils traceroute

Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
Package ntp-4.2.6p5-25.el7.centos.2.x86_64 already installed and latest version
Package 32:bind-utils-9.9.4-51.el7_4.2.x86_64 already installed and latest version
Package 3:traceroute-2.0.22-2.el7.x86_64 already installed and latest version
Nothing to do


In [7]:
systemctl enable ntpd

# adjust `/etc/ntp.conf` for the workshop NTP server
<p>review the contents of the file</p>

In [8]:
cat /etc/ntp.conf

# /etc/ntp.conf, configuration for NTP
# by default act only as a basic NTP client
restrict -4 default nomodify nopeer noquery notrap
restrict -6 default nomodify nopeer noquery notrap
# allow NTP messages from the loopback address, useful for debugging
restrict 127.0.0.1
restrict ::1
logfile /var/log/ntpd
driftfile /var/lib/ntp/ntp.drift
statsdir  /var/lib/ntp/
statistics loopstats peerstats clockstats
filegen loopstats file loopstats type day enable
filegen peerstats file peerstats type day enable
filegen clockstats file clockstats type day enable

# You should have at least 4 NTP servers

# server ntp-ucla.usno.navy.mil iburst   # Naval Observatory - Los Angeles, CA USA 

# server owamp.losa.net.internet2.edu iburst   # Internet2 - Los Angeles, CA USA 

# server owamp.salt.net.internet2.edu iburst   # Internet2 - Salt Lake City, UT USA 

# server saturn.es.net iburst   # ESnet - Sunnyvale, CA USA 

# server time3.chpc.utah.edu iburst   # UTAH - Server #3, University of Utah, Salt La

<p>using your favorite editor, comment out NTP servers outside of the workshop and add an entry for the local NTP server</p>
<pre>
#server time.apple.com
server 137.164.48.196 iburst
</pre>

## verify the changes

In [9]:
cat /etc/ntp.conf

# /etc/ntp.conf, configuration for NTP
# by default act only as a basic NTP client
restrict -4 default nomodify nopeer noquery notrap
restrict -6 default nomodify nopeer noquery notrap
# allow NTP messages from the loopback address, useful for debugging
restrict 127.0.0.1
restrict ::1
logfile /var/log/ntpd
driftfile /var/lib/ntp/ntp.drift
statsdir  /var/lib/ntp/
statistics loopstats peerstats clockstats
filegen loopstats file loopstats type day enable
filegen peerstats file peerstats type day enable
filegen clockstats file clockstats type day enable

# You should have at least 4 NTP servers

# server ntp-ucla.usno.navy.mil iburst   # Naval Observatory - Los Angeles, CA USA 

# server owamp.losa.net.internet2.edu iburst   # Internet2 - Los Angeles, CA USA 

# server owamp.salt.net.internet2.edu iburst   # Internet2 - Salt Lake City, UT USA 

# server saturn.es.net iburst   # ESnet - Sunnyvale, CA USA 

# server time3.chpc.utah.edu iburst   # UTAH - Server #3, University of Utah, Salt La

In [10]:
systemctl start ntpd

In [11]:
systemctl status ntpd -l

● ntpd.service - Network Time Service
   Loaded: loaded (/usr/lib/systemd/system/ntpd.service; enabled; vendor preset: disabled)
   Active: active (running) since Sat 2018-03-03 16:05:10 PST; 3h 30min ago
 Main PID: 1042 (ntpd)
   CGroup: /system.slice/ntpd.service
           └─1042 /usr/sbin/ntpd -u ntp:ntp -g

Mar 03 16:05:10 gigiot-22 systemd[1]: Starting Network Time Service...
Mar 03 16:05:10 gigiot-22 ntpd[1042]: proto: precision = 0.100 usec
Mar 03 16:05:10 gigiot-22 ntpd[1042]: 0.0.0.0 c01d 0d kern kernel time sync enabled
Mar 03 16:05:10 gigiot-22 systemd[1]: Started Network Time Service.


In [12]:
ntpq -p -c rv

     remote           refid      st t when poll reach   delay   offset  jitter
*fiona-10-01     69.89.207.99     2 u  269  512  377    0.113    2.004   0.867
associd=0 status=0615 leap_none, sync_ntp, 1 event, clock_sync,
version="ntpd 4.2.6p5@1.2349-o Wed Apr 12 21:24:06 UTC 2017 (1)",
processor="x86_64", system="Linux/3.10.0-693.17.1.el7.x86_64", leap=00,
stratum=3, precision=-23, rootdelay=67.801, rootdisp=44.224,
refid=137.164.48.196,
reftime=de45e6f7.a1fe44d9  Sat, Mar  3 2018 19:31:03.632,
clock=de45e805.758f3a33  Sat, Mar  3 2018 19:35:33.459, peer=43887, tc=9,
mintc=3, offset=2.004, frequency=-19.190, sys_jitter=0.000,
clk_jitter=5.157, clk_wander=1.103


## determine default interface

In [13]:
default_iface=$(awk '$2 == 00000000 { print $1 }' /proc/net/route)
echo $default_iface

enp3s0


## confirm `hostname -f` responds with FQHN

In [14]:
hostname -f

gigiot-22.conf.cenic.org


## use hostnamectl to set if its not a valid FQHN

In [15]:
myip=$(ip addr show dev $default_iface | grep 'inet '| awk '{print $2}' | awk -F "/" '{print $1}')
dig -x $myip +short

gigiot-22.


In [16]:
hostnamectl set-hostname `dig -x $myip +short`
hostname -f

gigiot-22.conf.cenic.org


## view current link settings on default interface

In [17]:
ip link show dev $default_iface

2: enp3s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq state UP mode DEFAULT qlen 10000
    link/ether 40:8d:5c:f4:b6:c6 brd ff:ff:ff:ff:ff:ff


## create a custom udev rules file for tuning the network configuration 

In [18]:
if [ ! -f /etc/udev/rules.d/80-prp-fiona-ws-ma-netconf.rules ]; then
cat > /etc/udev/rules.d/80-prp-fiona-ws-ma-netconf.rules <<EOL
KERNEL=="$default_iface", RUN+="/sbin/ip link set %k mtu 9000"
KERNEL=="$default_iface", RUN+="/sbin/ip link set %k txqueuelen 10000"
EOL
chmod 644 /etc/udev/rules.d/80-prp-fiona-ws-ma-netconf.rules
fi

## Use udevadm to reload the rules and trigger the action

In [19]:
/sbin/udevadm control --reload-rules
/sbin/udevadm trigger

## confirm desired settings are in effect

In [20]:
ip link show dev $default_iface

2: enp3s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq state UP mode DEFAULT qlen 10000
    link/ether 40:8d:5c:f4:b6:c6 brd ff:ff:ff:ff:ff:ff


## Create a custom sysctl file with desired settings

In [21]:
if [ ! -f /etc/sysctl.d/80-prp-fiona-ws-ma-sysctl.conf ]; then
cat>/etc/sysctl.d/80-prp-fiona-ws-ma-sysctl.conf <<EOL
net.core.rmem_max = 67108864 
net.core.wmem_max = 67108864 
net.ipv4.tcp_rmem = 4096 87380 33554432
net.ipv4.tcp_wmem = 4096 65536 33554432
net.ipv4.tcp_congestion_control=htcp
EOL
chmod 644 /etc/sysctl.d/80-prp-fiona-ws-ma-sysctl.conf
fi

## Adjust kernel boot-time defaults and rebuild grub.cfg

In [22]:
if ! grep --quiet default_hugepagesz /etc/default/grub; then
	sed -i 's/^GRUB_CMDLINE_LINUX=\"[^\"]*/& default_hugepagesz=1GB hugepagesz=1GB hugepages=8 elevator=noop processor.max_cstate=1 intel_iommu=off idle=poll intel_idle.max_cstate=0 nosoftlockup mce=ignore_mce/' /etc/default/grub
	grub2-mkconfig -o /boot/grub2/grub.cfg
fi

In [23]:
cat /etc/default/grub

GRUB_TIMEOUT=5
GRUB_DISTRIBUTOR="$(sed 's, release .*$,,g' /etc/system-release)"
GRUB_DEFAULT=saved
GRUB_DISABLE_SUBMENU=true
GRUB_TERMINAL_OUTPUT="console"
GRUB_CMDLINE_LINUX="crashkernel=auto rhgb quiet default_hugepagesz=1GB hugepagesz=1GB hugepages=8 elevator=noop processor.max_cstate=1 intel_iommu=off idle=poll intel_idle.max_cstate=0 nosoftlockup mce=ignore_mce"
GRUB_DISABLE_RECOVERY="true"


# SAVE your work NOW

In [ ]:
reboot

## confirm intended tuning settings are present 

In [1]:
ip link show | grep UP | grep -v lo

2: enp3s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq state UP mode DEFAULT qlen 10000


## disable a few unwanted services and enable the `tuned` service

In [2]:
systemctl stop NetworkManager 
systemctl disable NetworkManager
systemctl stop irqbalance.service
systemctl disable irqbalance.service
systemctl stop wpa_supplicant
systemctl disable wpa_supplicant
systemctl start tuned
systemctl enable tuned


## check the current clock rate of the CPU cores

In [3]:
cat /proc/cpuinfo | grep Hz

model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000
model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000
model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000
model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000


## use `tuned-adm` to show the current active profile

In [4]:
tuned-adm active

Current active profile: network-throughput


## set `throughput-performance` at the active profile

In [5]:
tuned-adm profile throughput-performance

## re-check the clock rate of the CPU cores

In [6]:
cat /proc/cpuinfo | grep Hz

model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000
model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000
model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000
model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000


# Installing the perfSONAR Central Management bundle
<p>The instructions below follow the perfSONAR CentOS Bundle Installation page, here:</p>
<p>http://docs.perfsonar.net/install_centos.html</p>

## point yum to local perfSONAR repo 

In [7]:
cat << EOF > /etc/yum.repos.d/perfsonar.repo
[perfsonar]
name=Extra Packages for Enterprise Linux 7 - $basearch
baseurl=http://137.164.48.196/centos-7-x86_64/perfsonar
failovermethod=priority
enabled=1
gpgcheck=0
EOF

## Refresh the yum cache 

In [8]:
yum clean all

Loaded plugins: fastestmirror
Cleaning repos: base epel extras globus perfsonar updates zfs
Cleaning up everything
Maybe you want: rm -rf /var/cache/yum, to also free up space taken by orphaned data from disabled or removed repos
Cleaning up list of fastest mirrors


## Now we are ready to install the `perfsonar-centralmanagement` bundle:
<p>due to the large number of packages involved, the call to yum in the next cell includes the `-q` switch to set yum to quiet mode.  output to `stderr` and any non-zero exit code will appear in the cell.</p>
<p>NOTE: launch a separate terminal session and run <pre>tail -f /var/log/yum.log</pre> to follow the installation process.</p>

In [9]:
yum -y -q install perfsonar-centralmanagement

Package perfsonar-centralmanagement-4.0.2.1-1.el7.centos.noarch already installed and latest version


## use `systemctl` to enable services

In [10]:
systemctl enable httpd
systemctl start httpd
systemctl enable cassandra
systemctl start cassandra

cassandra.service is not a native service, redirecting to /sbin/chkconfig.
Executing /sbin/chkconfig cassandra on


## check firewall rules

In [11]:
firewall-cmd --list-all

public (active)
  target: default
  icmp-block-inversion: no
  interfaces: enp3s0
  sources: 
  services: ssh dhcpv6-client ntp http https traceroute bwctl owamp-control oppd
  ports: 5001-5300/udp 5001-5300/tcp 5301-5600/udp 5301-5600/tcp 5601-5900/udp 5601-5900/tcp 6001-6200/udp 6001-6200/tcp 8760-9960/udp 8760-9960/tcp 8888/tcp 2811/tcp 7512/tcp 50000-51000/tcp 50000-51000/udp 2223/tcp 4823/tcp 5001-5900/tcp 5001-5900/udp
  protocols: 
  masquerade: no
  forward-ports: 
  source-ports: 
  icmp-blocks: 
  rich rules: 
	


## configure the firewall rules to allow http / https

In [12]:
firewall-cmd --permanent --zone=public --add-service=http
firewall-cmd --permanent --zone=public --add-service=https
firewall-cmd --reload

success
success
success


## verify updates to the firewall ruleset

In [13]:
firewall-cmd --list-all

public (active)
  target: default
  icmp-block-inversion: no
  interfaces: enp3s0
  sources: 
  services: ssh dhcpv6-client ntp http https traceroute bwctl owamp-control oppd
  ports: 5001-5300/udp 5001-5300/tcp 5301-5600/udp 5301-5600/tcp 5601-5900/udp 5601-5900/tcp 6001-6200/udp 6001-6200/tcp 8760-9960/udp 8760-9960/tcp 8888/tcp 2811/tcp 7512/tcp 50000-51000/tcp 50000-51000/udp 2223/tcp 4823/tcp 5001-5900/tcp 5001-5900/udp
  protocols: 
  masquerade: no
  forward-ports: 
  source-ports: 
  icmp-blocks: 
  rich rules: 
	


## SAVE your work NOW
## Reboot the server 


In [ ]:
reboot

## verify things came up as expected

In [4]:
systemctl status maddash-server -l
sudo systemctl restart maddash-server
systemctl status perfsonar-meshconfig-guiagent -l
systemctl status postgresql-9.5 -l
systemctl status cassandra -l
systemctl status httpd -l

● maddash-server.service - MaDDash Scheduler and REST Server
   Loaded: loaded (/usr/lib/systemd/system/maddash-server.service; enabled; vendor preset: disabled)
   Active: active (running) since Sat 2018-03-03 19:48:15 PST; 8s ago
  Process: 6451 ExecStart=/bin/sh -c /usr/lib/maddash/maddash-server/bin/startServer.sh /var/run/maddash/maddash-server.pid /usr/lib/maddash/maddash-server/target/maddash-server.one-jar.jar -c /etc/maddash/maddash-server/maddash.yaml -l /etc/maddash/maddash-server/log4j.properties >/var/log/maddash/maddash-server.out 2>&1 (code=exited, status=0/SUCCESS)
  Process: 6448 ExecStartPre=/bin/chown -R maddash:maddash /var/log/maddash (code=exited, status=0/SUCCESS)
  Process: 6445 ExecStartPre=/bin/chown maddash:maddash /var/run/maddash (code=exited, status=0/SUCCESS)
  Process: 6443 ExecStartPre=/bin/mkdir -p /var/run/maddash (code=exited, status=0/SUCCESS)
 Main PID: 6455 (java)
   CGroup: /system.slice/maddash-server.service
           └─6455 java -Djava.net.pr

## verify the MaDDash webUI is available
<p>the following cell will make a call to `curl` to request the text of the MaDDash webUI main page.</p>
<p>alternatively, launch a new web browser tab to view the dashboard.</p>

In [5]:
curl http://$(hostname -s)/maddash-webui/

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html>
	<head>
		<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">
		<title>MaDDash - Monitoring and Debugging Dashboard</title>
		<link rel="stylesheet" href="lib/dojo/dijit/themes/claro/claro.css" media="screen">
		<link rel="stylesheet" href="style/maddash-webui.css" media="screen">
		<link rel="stylesheet" href="style/maddash.css" media="screen">
		<link rel="stylesheet" href="style/tipsy.css" />
        <link rel="stylesheet" href="style/jquery.fancybox-1.3.4.css" />
	
	    <script src="lib/jquery.min.js"></script>
        <script src="lib/jquery-ui.min.js"></script>
        <script type="text/javascript" src="lib/jquery.tipsy.js"></script>
		<script src="lib/dojo/dojo/dojo.js" data-dojo-config="parseOnLoad: true"></script>
		<script src="lib/dojo/dojo/dojo-maddash.js"></script>
		<script src="lib/d3.v2.js"></script>
		<script type="text/javascript" src="lib/maddash.js

## verify the esmond MA is operational
<p>without any data in the MA at this point the expected output is an open and close pair of square brackets</p>

In [6]:
curl http://$(hostname -s)/esmond/perfsonar/archive/?format=json

[{"url":"http://gigiot-22/esmond/perfsonar/archive/0263c9e018b3499baf6f437ab05f9c43/?format=json","metadata-key":"0263c9e018b3499baf6f437ab05f9c43","subject-type":"point-to-point","event-types":[{"summaries":[],"time-updated":1520128226,"base-uri":"/esmond/perfsonar/archive/0263c9e018b3499baf6f437ab05f9c43/packet-retransmits/base","event-type":"packet-retransmits"},{"summaries":[],"time-updated":1520128226,"base-uri":"/esmond/perfsonar/archive/0263c9e018b3499baf6f437ab05f9c43/pscheduler-run-href/base","event-type":"pscheduler-run-href"},{"summaries":[{"summary-type":"average","uri":"/esmond/perfsonar/archive/0263c9e018b3499baf6f437ab05f9c43/throughput/averages/86400","time-updated":1520128226,"summary-window":"86400"}],"time-updated":1520128226,"base-uri":"/esmond/perfsonar/archive/0263c9e018b3499baf6f437ab05f9c43/throughput/base","event-type":"throughput"},{"summaries":[],"time-updated":null,"base-uri":"/esmond/perfsonar/archive/0263c9e018b3499baf6f437ab05f9c43/failures/base","event-t

## create an adminstrative user for the MaDDash webUI
<p>MaDDash provides an adminstrative interface available through the webUI at <pre>http://enter.the.ma.ip.here/maddash-webui/admin</pre> <p>in a termal session, use `htpasswd` to update `/etc/maddash/maddash-webui/admin-users`</p>
<pre>htpasswd /etc/maddash/maddash-webui/admin-users admin</pre>

# Creating and publishing a perfSONAR mesh json
<p>the next set of steps will produce and publish an updated mesh json:
<ul>
<li>process the mesh json to update MaDDash server yaml configuration: configuring checks to query the esmond; thresholds for performance, and organizing dashboards</li>
<li>publish the mesh json for participating perfSONAR nodes: which process the mesh json to update their test configuration</li>
</ul>
</p>

## use wget to transfer a baseline mesh configuration (.conf)
<p>the mesh .conf file provides a more human-readable configuration syntax (than json).</p>
<p>subsequent steps reference the mesh .conf to build the mesh json</p>

In [7]:
cd /var/www/html
wget http://fiona-10-01:8000/prp-fiona-ws-mesh--$(hostname -s).conf -O prp-fiona-ws-mesh--$(hostname -s).conf
curl -v http://gigiot-ma-4/prp-fiona-ws-mesh--gigiot-ma-4.json

--2018-03-03 19:51:12--  http://fiona-10-01:8000/prp-fiona-ws-mesh--gigiot-22.conf
Resolving fiona-10-01 (fiona-10-01)... 137.164.48.196
Connecting to fiona-10-01 (fiona-10-01)|137.164.48.196|:8000... connected.
HTTP request sent, awaiting response... 404 File not found
2018-03-03 19:51:12 ERROR 404: File not found.

* About to connect() to gigiot-ma-4 port 80 (#0)
*   Trying 137.164.48.232...
* Connected to gigiot-ma-4 (137.164.48.232) port 80 (#0)
> GET /prp-fiona-ws-mesh--gigiot-ma-4.json HTTP/1.1
> User-Agent: curl/7.29.0
> Host: gigiot-ma-4
> Accept: */*
> 
< HTTP/1.1 200 OK
< Date: Sun, 04 Mar 2018 03:51:12 GMT
< Server: Apache/2.4.6 (CentOS) OpenSSL/1.0.2k-fips mod_wsgi/3.4 Python/2.7.5
< Last-Modified: Sat, 03 Mar 2018 23:35:17 GMT
< ETag: "368e-5668a8ba2e2d3"
< Accept-Ranges: bytes
< Content-Length: 13966
< Content-Type: application/json
< 
{
   "administrators" : [
      {
         "email" : "jhess@cenic.org",
         "name" : "John Hess"
      }
   ],
   "organizations" : [

## check the contents of the baseline mesh .conf
<p>use your favorite editor to make any necessary changes</p>

In [9]:
cat /var/www/html/prp-fiona-ws-mesh--$(hostname -s).conf

## generate the perfSONAR mesh json

In [10]:
/usr/lib/perfsonar/bin/build_json --verbose /var/www/html/prp-fiona-ws-mesh--$(hostname -s).conf > /var/www/html/prp-fiona-ws-mesh--$(hostname -s).json

## view the mesh json

In [11]:
cat /var/www/html/prp-fiona-ws-mesh--$(hostname -s).json

{}


## validate the generated perfSONAR mesh json

In [12]:
/usr/lib/perfsonar/bin/validate_json --verbose --config_url=http://$(hostname -s)/prp-fiona-ws-mesh--$(hostname -s).json

2018/03/03 19:57:09 (8833) DEBUG> Utils.pm:119 perfSONAR_PS::Client::Utils::send_http_request - Sending HTTP GET to http://gigiot-22/prp-fiona-ws-mesh--gigiot-22.json


# Update the MaDDash server yaml configuration
<p>in the next series of steps we will:</p>
<ul>
<li>set up a fresh `maddash.yaml` configuration on which to subsequently apply changes</li>
<li>update the `meshconfig-guiagent.conf` file to create a mesh referencing the newly created perfSONAR mesh json</li>
<li>update the `maddash.yaml` for GridFTP checks, thresholds, and dashboards</li>
</ul>

## create a fresh `maddash.yaml` file

In [ ]:
if [ -f /etc/maddash/maddash-server/maddash.yaml ]; then
mv /etc/maddash/maddash-server/maddash.yaml /etc/maddash/maddash-server/maddash.yaml.orig
cat>/etc/maddash/maddash-server/maddash.yaml <<EOL
##
# Set the directory where the database will be stored
database: /var/lib/maddash/

##
# Set the number of jobs that can run in parallel. Default is 20.
#jobThreadPoolSize: 20

##
# Set number of jobs that can be in queue at one time. Default is 250.
#jobBatchSize: 250

###
# Disable the job scheduler if you only want to run the REST server
#disableScheduler: 0

###
# Skips table and index rebuild at start-up. It can speed up start-up time if set to 1.
#skipTableBuild: 0

##
# Set the host where the REST server listens
serverHost: "localhost"

##
# Activate http and set the port where it listens
http:
    port: 8881
EOL
chown maddash:maddash /etc/maddash/maddash-server/maddash.yaml
chmod 644 /etc/maddash/maddash-server/maddash.yaml
fi

## view the `maddash.yaml` file

In [ ]:
cat /etc/maddash/maddash-server/maddash.yaml

## update `meshconfig-guiagent.conf` to add a `<mesh>` stanza referencing the mesh json</p>

In [ ]:
if [ ! -f /etc/perfsonar/meshconfig-guiagent.conf.orig ]; then
cp /etc/perfsonar/meshconfig-guiagent.conf /etc/perfsonar/meshconfig-guiagent.conf.orig
cat >> /etc/perfsonar/meshconfig-guiagent.conf <<EOL
## You can define more meshes to configure against by adding more 'mesh' blocks.
<mesh>
    configuration_url             http://$(hostname -s)/prp-fiona-ws-mesh--$(hostname -s).json
#    #validate_certificate         0
#    #ca_certificate_file          /etc/pki/tls/certs/ca-bundle.crt
</mesh>
EOL
elif [ -f /etc/perfsonar/meshconfig-guiagent.conf.orig ]; then
rm -f /etc/perfsonar/meshconfig-guiagent.conf
cp /etc/perfsonar/meshconfig-guiagent.conf.orig /etc/perfsonar/meshconfig-guiagent.conf
cat >> /etc/perfsonar/meshconfig-guiagent.conf <<EOL
## You can define more meshes to configure against by adding more 'mesh' blocks.
<mesh>
    configuration_url             http://$(hostname -s)/prp-fiona-ws-mesh--$(hostname -s).json
#    #validate_certificate         0
#    #ca_certificate_file          /etc/pki/tls/certs/ca-bundle.crt
</mesh>
EOL
fi

## review the updated `meshconfig-guiagent.conf`

In [ ]:
cat /etc/perfsonar/meshconfig-guiagent.conf 

## restart the `perfsonar-meshconfig-guiagent` process

In [ ]:
systemctl restart perfsonar-meshconfig-guiagent

## review the updated `maddash.yaml`

In [ ]:
cat /etc/maddash/maddash-server/maddash.yaml

## restart the `maddash-server` process


In [ ]:
systemctl restart maddash-server

## browse to the MaDDAsh server webUI to confirm the updates

# Update MaDDash server configuration file for visualizing GridFTP disk-to-disk throughput  


## apply a patch file to a copy of `maddash.yaml` to insert sections for GridFTP-related checks, dashboards and grids, and groups

In [ ]:
cd /etc/maddash/maddash-server/
cat maddash.yaml > maddash--$(hostname -s)--ps.yaml
wget http://fiona-10-01:8000/maddash--$(hostname -s)--ps-gridftp.yaml.patch -O maddash--$(hostname -s)--ps-gridftp.yaml.patch
patch --input=maddash--$(hostname -s)--ps-gridftp.yaml.patch --output=maddash--$(hostname -s)--ps-gridftp.yaml maddash--$(hostname -s)--ps.yaml

## sanity check the diff of the copy of `maddash.yaml` and patched file

In [ ]:
diff -U 4 maddash--$(hostname -s)--ps.yaml maddash--$(hostname -s)--ps-gridftp.yaml

## replace `maddash.yaml` with the patched file

In [ ]:
cat maddash--$(hostname -s)--ps-gridftp.yaml > maddash.yaml

# update the MaDDash webUI `config.json` to reflect the customized dashboard naming
<p>review the current contents of <pre>/etc/maddash/maddash-webui/config.json</pre></p>

In [ ]:
cd /etc/maddash/maddash-webui
cat config.json

## use `wget` to transfer an customized `config.json` and replace the original

In [ ]:
wget http://fiona-10-01:8000/webui--$(hostname -s)--config.json -O webui--$(hostname -s)--config.json
cat webui--$(hostname -s)--config.json > config.json

## restart the `maddash-server` process 
<p>NOTE: in a separate terminal session, run <pre>tail -f /var/log/maddash/maddash-server.netlogger.log</pre> to check for errors</p>

In [ ]:
systemctl restart maddash-server

## browse to the MaDDash webUI to confirm the new GridFTP dashboards are present
<p>NOTE: you will need to clear your browsers cache in order to view the results of the updated webUI config.json</p>
<p>ex: with Chrome, select <pre>Clear Browsing Data...</pre> to bring up a dialog of options.  clearing the last hour of browsing data should be sufficient</p>

# Update esmond to add credentials to register GridFTP transfer results
<p>the following steps will update your central esmond MA to authtenticate GridFTP nodes by an API key</p>  
<p>on the nodes running GridFTP transfers, we will use the generated API key to configure the script which parses the transfer logs and registers the throughput results to the esmond MA</p>
<p>NOTE: you may re-run this cell and it will display the previously generated API key</p>


In [ ]:
cd /usr/lib/esmond
. bin/activate
python esmond/manage.py add_api_key_user gridftp

# Update esmond to add credentials for pS nodes to register results
<p>the following cell will update your central esmond MA to authtenticate pS nodes to by their IP address</p>

In [ ]:
cd /usr/lib/esmond
. bin/activate
for node in $(cat /var/www/html/prp-fiona-ws-mesh--$(hostname -s).conf | grep member | awk '{ print $2 }'); do  python esmond/manage.py add_user_ip_address $node `dig +short $node` ; done


# Update nodes to schedule GridFTP transfers and perfSONAR tests, and register results to the central MA 
<p>in the next session, we will work on the nodes running GridFTP server and perfSONAR testpoint to update their configurations</p>
<p>this would be a good time to SAVE your WORK in this notebook</p>